In [1]:
# ============================================================================
# Promotion Lift Modeling with Neural Networks
# Topic: Non-linear promotional effects, cannibalization, ROI estimation
# Input: Sales time-series with promotions, channels, timing, discounts
# Output: Baseline vs uplift predictions, interaction effects, ROI metrics
# ============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import warnings
warnings.filterwarnings('ignore')

In [2]:
# ==== 1. CONFIG & CONSTANTS ====

DATA_PATH = "synthetic_promo_data.csv"
RANDOM_SEED = 42
TEST_SIZE = 0.2
VALIDATION_SPLIT = 0.15

# Model hyperparameters
NN_HIDDEN_UNITS = [64, 32, 16]
NN_LEARNING_RATE = 0.001
NN_EPOCHS = 100
NN_BATCH_SIZE = 32

# Business parameters
PRODUCT_COST = 15.0  # $ per unit
MIN_DISCOUNT_THRESHOLD = 0.05  # 5% minimum discount to trigger promo
CANNIBALIZATION_THRESHOLD = -0.10  # -10% = cannibalization signal

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [3]:
# ==== 2. SYNTHETIC DATA GENERATION ====

def generate_promotion_data(n_rows=500):
    """
    Create realistic promotional sales data with:
    - Baseline demand (day-of-week, trend)
    - Promotional effects (discount-driven lift)
    - Channel differences (online vs. retail)
    - Interaction effects (discount × channel × timing)
    - Cannibalization (weekend discount cannibalizes nearby weekday sales)
    """
    np.random.seed(RANDOM_SEED)
    
    data = {
        'day_of_week': np.random.randint(0, 7, n_rows),
        'hour': np.random.randint(0, 24, n_rows),
        'channel': np.random.choice(['online', 'retail'], n_rows),
        'discount_pct': np.random.uniform(0, 50, n_rows),
        'is_weekend': np.random.choice([0, 1], n_rows),
        'competitor_discount': np.random.uniform(0, 40, n_rows),
        'inventory_level': np.random.uniform(0.2, 1.0, n_rows),
    }
    
    df = pd.DataFrame(data)
    
    # Baseline: day-of-week pattern
    dow_baseline = np.array([80, 85, 90, 95, 120, 110, 100])
    df['baseline_demand'] = df['day_of_week'].map(lambda x: dow_baseline[x])
    
    # Discount lift (non-linear: diminishing returns at high discounts)
    df['discount_effect'] = df['discount_pct'] * 0.8 - (df['discount_pct'] ** 2) * 0.005
    
    # Channel interaction: online responds stronger to discounts
    channel_multiplier = np.where(df['channel'] == 'online', 1.5, 1.0)
    df['channel_interaction'] = df['discount_effect'] * channel_multiplier
    
    # Weekend cannibalization: high weekend discount hurts future demand
    df['cannibalization'] = np.where(
        (df['is_weekend'] == 1) & (df['discount_pct'] > 20),
        -0.15 * df['discount_pct'],
        0
    )
    
    # Competitive pressure (reduce lift if competitor also discounts)
    df['competitive_effect'] = -0.3 * np.minimum(df['discount_pct'], df['competitor_discount'])
    
    # Total sales (baseline + promotional + cannibalization)
    df['sales_units'] = (
        df['baseline_demand'] +
        df['channel_interaction'] +
        df['cannibalization'] +
        df['competitive_effect'] +
        np.random.normal(0, 5, n_rows)  # noise
    )
    
    df['sales_units'] = np.maximum(df['sales_units'], 10)  # floor at 10
    df['sales_revenue'] = df['sales_units'] * PRODUCT_COST
    
    # Encode channel as numeric
    df['channel_online'] = (df['channel'] == 'online').astype(int)
    
    return df

In [4]:
# ==== 3. FEATURE ENGINEERING ====

def engineer_features(df):
    """
    Prepare features for modeling:
    - Scale continuous variables
    - Create interaction terms (discount × channel, discount × weekend)
    - Add lag features (previous day demand for cannibalization detection)
    """
    df_feat = df.copy()
    
    # Interaction terms
    df_feat['discount_x_online'] = df_feat['discount_pct'] * df_feat['channel_online']
    df_feat['discount_x_weekend'] = df_feat['discount_pct'] * df_feat['is_weekend']
    df_feat['discount_x_hour'] = df_feat['discount_pct'] * (df_feat['hour'] / 24.0)
    
    # Create lag demand (for cannibalization analysis)
    df_feat['sales_lag1'] = df_feat['sales_units'].shift(1).fillna(df_feat['sales_units'].mean())
    
    # Polynomial features for non-linearity
    df_feat['discount_sq'] = df_feat['discount_pct'] ** 2
    df_feat['discount_cube'] = df_feat['discount_pct'] ** 3
    
    return df_feat

In [5]:
# ==== 4. LINEAR BASELINE MODEL ====

def train_linear_baseline(X_train, y_train, X_test, y_test):
    """
    Fit simple linear regression as baseline for comparison.
    Captures marginal effects but misses non-linearities & interactions.
    """
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    print("=== LINEAR BASELINE ===")
    print(f"RMSE (train): {rmse_train:.2f}")
    print(f"RMSE (test):  {rmse_test:.2f}")
    print()
    
    return model, y_pred_test

In [6]:
# ==== 5. NEURAL NETWORK MODEL ====

def build_nn_model(input_dim):
    """
    Build deep neural network for non-linear promotional effects.
    Architecture: Input → Dense layers with ReLU → Output (linear)
    """
    model = keras.Sequential([
        layers.Dense(NN_HIDDEN_UNITS[0], activation='relu', input_dim=input_dim),
        layers.Dropout(0.2),
        layers.Dense(NN_HIDDEN_UNITS[1], activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(NN_HIDDEN_UNITS[2], activation='relu'),
        layers.Dense(1, activation='linear')  # Regression output
    ])
    
    optimizer = keras.optimizers.Adam(learning_rate=NN_LEARNING_RATE)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model

def train_nn_model(X_train, y_train, X_test, y_test, X_val, y_val):
    """
    Train neural network with validation monitoring.
    Returns model and training history.
    """
    model = build_nn_model(X_train.shape[1])
    
    history = model.fit(
        X_train, y_train,
        epochs=NN_EPOCHS,
        batch_size=NN_BATCH_SIZE,
        validation_data=(X_val, y_val),
        verbose=0
    )
    
    y_pred_test = model.predict(X_test, verbose=0).flatten()
    
    rmse_train = np.sqrt(mean_squared_error(y_train, model.predict(X_train, verbose=0).flatten()))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    print("=== NEURAL NETWORK MODEL ===")
    print(f"RMSE (train): {rmse_train:.2f}")
    print(f"RMSE (test):  {rmse_test:.2f}")
    print()
    
    return model, y_pred_test, history

In [7]:
# ==== 6. BASELINE vs. UPLIFT MODELING ====

def estimate_baseline_demand(model_nn, df_test):
    """
    Estimate baseline (no promotion) demand by setting discount_pct = 0.
    Compare vs. actual sales to isolate promotional lift.
    """
    X_baseline = df_test.copy()
    
    # Zero out all discount-related features
    discount_cols = [c for c in X_baseline.columns if 'discount' in c.lower()]
    for col in discount_cols:
        X_baseline[col] = 0.0
    
    baseline_pred = model_nn.predict(X_baseline, verbose=0).flatten()
    return baseline_pred

def calculate_uplift(actual_sales, baseline_pred, model_pred):
    """
    Compute incremental lift:
    - Gross Uplift = Actual - Baseline
    - Net Uplift = Model Pred - Baseline (accounts for interaction effects)
    """
    gross_uplift = actual_sales - baseline_pred
    net_uplift = model_pred - baseline_pred
    uplift_pct = (gross_uplift / baseline_pred) * 100
    
    return pd.DataFrame({
        'baseline': baseline_pred,
        'model_pred': model_pred,
        'actual': actual_sales,
        'gross_uplift': gross_uplift,
        'net_uplift': net_uplift,
        'uplift_pct': uplift_pct
    })

In [8]:
# ==== 7. INTERACTION EFFECTS ANALYSIS ====

def analyze_interactions(df_test, uplift_df):
    """
    Break down lift by discount × channel × timing combinations.
    Shows which segments benefit most from promotions.
    """
    results = df_test[['discount_pct', 'channel_online', 'is_weekend']].copy()
    results['uplift_pct'] = uplift_df['uplift_pct']
    
    # Interaction by channel
    print("=== INTERACTION EFFECTS ===")
    print("\nBy Channel:")
    for channel in [0, 1]:
        mask = results['channel_online'] == channel
        avg_uplift = results[mask]['uplift_pct'].mean()
        channel_name = 'Online' if channel == 1 else 'Retail'
        print(f"  {channel_name:8s}: {avg_uplift:6.2f}%")
    
    # Interaction by discount level
    print("\nBy Discount Level:")
    results['discount_bin'] = pd.cut(results['discount_pct'], bins=3, labels=['Low', 'Med', 'High'])
    for bin_name in ['Low', 'Med', 'High']:
        mask = results['discount_bin'] == bin_name
        avg_uplift = results[mask]['uplift_pct'].mean()
        print(f"  {bin_name:8s}: {avg_uplift:6.2f}%")
    
    # Interaction by timing (weekend vs. weekday)
    print("\nBy Timing:")
    for is_wknd in [0, 1]:
        mask = results['is_weekend'] == is_wknd
        avg_uplift = results[mask]['uplift_pct'].mean()
        timing_name = 'Weekend' if is_wknd == 1 else 'Weekday'
        print(f"  {timing_name:8s}: {avg_uplift:6.2f}%")
    
    print()
    return results

In [9]:
# ==== 8. CANNIBALIZATION DETECTION ====

def detect_cannibalization(df_test, uplift_df):
    """
    Flag potential cannibalization: high discount × is_weekend with negative uplift.
    Cannibalization = promo on weekend steals sales from weekday.
    """
    results = df_test[['discount_pct', 'is_weekend', 'sales_units']].copy()
    results['uplift_pct'] = uplift_df['uplift_pct']
    
    # Identify high-discount weekend promos with poor uplift
    cannibal_mask = (
        (results['discount_pct'] > 25) &
        (results['is_weekend'] == 1) &
        (results['uplift_pct'] < CANNIBALIZATION_THRESHOLD * 100)
    )
    
    cannibal_count = cannibal_mask.sum()
    cannibal_avg_uplift = results[cannibal_mask]['uplift_pct'].mean() if cannibal_count > 0 else 0
    
    print("=== CANNIBALIZATION DETECTION ===")
    print(f"High-discount weekend promos flagged: {cannibal_count}/{len(results)}")
    print(f"Avg uplift (cannibalized): {cannibal_avg_uplift:.2f}%")
    print()
    
    return results[cannibal_mask]

In [10]:
# ==== 9. ROI & INCREMENTAL DEMAND ESTIMATION ====

def calculate_promo_roi(df_test, uplift_df):
    """
    Compute ROI per promotion:
    - Gross Revenue = Uplift × Price
    - Promo Cost = Discount Amount (foregone margin)
    - Net Contribution = Gross Revenue - Promo Cost
    - ROI = Net Contribution / Promo Cost
    """
    roi_results = df_test[['discount_pct', 'sales_units', 'channel']].copy()
    roi_results['baseline'] = uplift_df['baseline']
    roi_results['uplift_units'] = uplift_df['gross_uplift']
    
    # Promo cost = discount applied to baseline volume
    roi_results['promo_cost'] = (roi_results['baseline'] * roi_results['discount_pct'] / 100) * PRODUCT_COST
    
    # Incremental revenue = uplift × price
    roi_results['incremental_revenue'] = roi_results['uplift_units'] * PRODUCT_COST
    
    # Net contribution (revenue - cost)
    roi_results['net_contribution'] = roi_results['incremental_revenue'] - roi_results['promo_cost']
    
    # ROI = net / cost (avoid division by zero)
    roi_results['roi'] = np.where(
        roi_results['promo_cost'] > 1,
        (roi_results['net_contribution'] / roi_results['promo_cost']) * 100,
        0
    )
    
    print("=== ROI & INCREMENTAL DEMAND ===")
    print(f"Avg uplift units per promotion: {roi_results['uplift_units'].mean():.1f}")
    print(f"Avg incremental revenue per promo: ${roi_results['incremental_revenue'].mean():.2f}")
    print(f"Avg ROI: {roi_results['roi'].mean():.1f}%")
    
    # ROI by channel
    print("\nROI by Channel:")
    for channel in ['online', 'retail']:
        mask = roi_results['channel'] == channel
        avg_roi = roi_results[mask]['roi'].mean()
        print(f"  {channel.capitalize():8s}: {avg_roi:6.1f}%")
    
    print()
    return roi_results

In [11]:
# ==== 10. VISUALIZATION ====

def plot_results(y_test, y_pred_linear, y_pred_nn, history):
    """
    Compare model predictions and show training convergence.
    """
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    
    # Model comparison
    axes[0, 0].scatter(y_test, y_pred_linear, alpha=0.5, label='Linear', s=20)
    axes[0, 0].scatter(y_test, y_pred_nn, alpha=0.5, label='Neural Net', s=20)
    axes[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    axes[0, 0].set_xlabel('Actual Sales')
    axes[0, 0].set_ylabel('Predicted Sales')
    axes[0, 0].set_title('Model Predictions vs. Actual')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Residuals: Linear
    residuals_linear = y_test - y_pred_linear
    axes[0, 1].scatter(y_pred_linear, residuals_linear, alpha=0.5, s=20)
    axes[0, 1].axhline(0, color='r', linestyle='--')
    axes[0, 1].set_xlabel('Predicted Sales (Linear)')
    axes[0, 1].set_ylabel('Residuals')
    axes[0, 1].set_title('Linear Model Residuals')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Residuals: Neural Net
    residuals_nn = y_test - y_pred_nn
    axes[1, 0].scatter(y_pred_nn, residuals_nn, alpha=0.5, s=20)
    axes[1, 0].axhline(0, color='r', linestyle='--')
    axes[1, 0].set_xlabel('Predicted Sales (NN)')
    axes[1, 0].set_ylabel('Residuals')
    axes[1, 0].set_title('Neural Network Residuals')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Training history
    axes[1, 1].plot(history.history['loss'], label='Train Loss')
    axes[1, 1].plot(history.history['val_loss'], label='Val Loss')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss (MSE)')
    axes[1, 1].set_title('NN Training Convergence')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('promo_lift_results.png', dpi=100)
    print("Plot saved to 'promo_lift_results.png'\n")
    plt.close()

In [12]:
# ==== 11. MAIN PIPELINE ====

def main():
    """
    End-to-end promotional lift modeling:
    1. Generate synthetic data
    2. Feature engineering
    3. Train linear & NN models
    4. Compare baseline vs. uplift
    5. Analyze interactions & cannibalization
    6. Estimate ROI
    """
    
    print("Generating promotion data...")
    df = generate_promotion_data(n_rows=500)
    
    print("Engineering features...")
    df = engineer_features(df)
    
    # Prepare features and target
    feature_cols = [
        'day_of_week', 'hour', 'channel_online', 'discount_pct',
        'is_weekend', 'competitor_discount', 'inventory_level',
        'discount_x_online', 'discount_x_weekend', 'discount_x_hour',
        'sales_lag1', 'discount_sq', 'discount_cube'
    ]
    
    X = df[feature_cols].values
    y = df['sales_units'].values
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Train-test-val split
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_scaled, y, test_size=0.3, random_state=RANDOM_SEED
    )
    X_test, X_val, y_test, y_val = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=RANDOM_SEED
    )
    
    print("\n" + "="*50)
    print("MODEL TRAINING")
    print("="*50 + "\n")
    
    # Linear baseline
    model_linear, y_pred_linear = train_linear_baseline(X_train, y_train, X_test, y_test)
    
    # Neural network
    model_nn, y_pred_nn, history = train_nn_model(
        X_train, y_train, X_test, y_test, X_val, y_val
    )
    
    print("="*50)
    print("PROMOTIONAL ANALYSIS")
    print("="*50 + "\n")
    
    # Reconstruct test data with original features for analysis
    df_test = df.iloc[len(X_train) + len(X_val):len(X_train) + len(X_val) + len(X_test)].reset_index(drop=True)
    
    # Baseline vs uplift (NN predictions)
    baseline_pred = estimate_baseline_demand(model_nn, df_test[feature_cols])
    uplift_df = calculate_uplift(y_test, baseline_pred, y_pred_nn)
    
    print("Sample uplift breakdown (first 5 rows):")
    print(uplift_df.head())
    print()
    
    # Interaction effects
    interaction_results = analyze_interactions(df_test, uplift_df)
    
    # Cannibalization detection
    cannibal_df = detect_cannibalization(df_test, uplift_df)
    
    # ROI calculation
    roi_df = calculate_promo_roi(df_test, uplift_df)
    
    print("="*50)
    print("VISUALIZATION")
    print("="*50 + "\n")
    
    plot_results(y_test, y_pred_linear, y_pred_nn, history)
    
    print("="*50)
    print("DONE")
    print("="*50 + "\n")
    
    # TODO: Experiment with changing discount distribution or cannibalization threshold
    # and re-run to see how ROI and uplift shift. Try changing CANNIBALIZATION_THRESHOLD
    # to -0.05 (more aggressive) or -0.15 (more lenient) and observe results.
    
    return model_nn, scaler, roi_df

In [13]:
if __name__ == "__main__":
    model_nn, scaler, roi_results = main()
    print("\n✓ Promotion lift modeling complete!")
    print("✓ Try changing NN_HIDDEN_UNITS, NN_EPOCHS, or discount thresholds to experiment.")

Generating promotion data...
Engineering features...

MODEL TRAINING

=== LINEAR BASELINE ===
RMSE (train): 9.69
RMSE (test):  8.56

=== NEURAL NETWORK MODEL ===
RMSE (train): 10.30
RMSE (test):  10.02

PROMOTIONAL ANALYSIS

Sample uplift breakdown (first 5 rows):
      baseline  model_pred      actual  gross_uplift   net_uplift  uplift_pct
0  1756.620117   91.222252   84.474998  -1672.145119 -1665.397827  -95.191049
1  1544.960938  107.017921  110.707853  -1434.253085 -1437.942993  -92.834262
2  1558.845459   99.962936  117.214054  -1441.631405 -1458.882568  -92.480714
3  1559.461304  117.281708  132.766296  -1426.695007 -1442.179565  -91.486400
4  1783.564941   99.878113  101.787420  -1681.777521 -1683.686768  -94.293035

=== INTERACTION EFFECTS ===

By Channel:
  Retail  : -93.01%
  Online  : -92.88%

By Discount Level:
  Low     : -93.07%
  Med     : -93.17%
  High    : -92.58%

By Timing:
  Weekday : -92.71%
  Weekend : -93.10%

=== CANNIBALIZATION DETECTION ===
High-discount week